In [1]:
import sgml, sgutil, sgpp, dproc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import root_mean_squared_error

from proc_v2 import p
from ml_v2 import X_cat, X_num, X_all, target, kf, skf, ss, config
from ml_v2 import xgb_adapter, lgb_adapter, cb_adapter, lr_adapter, nn_adapter

sc = sgutil.SGCache('img', 'result', 'model')
df_train = p.fit_transform(['data/train.csv']).assign(
    Calories_Log = lambda x: np.log(x['Calories'] + 1)
)
df_test = p.transform(['data/test.csv'])

2025-05-20 05:20:27.784194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747718427.795855    4658 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747718427.799461    4658 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747718427.808510    4658 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747718427.808523    4658 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747718427.808525    4658 computation_placer.cc:177] computation placer alr

In [2]:
df_train['Duration'].nunique()

30

In [3]:
df_train['Heart_Rate'].nunique()

63

In [4]:
df_train['Body_Temp'].nunique()

75

# Target Encoder의 효과를 봅니다.

In [5]:
df_train.groupby(['Duration', 'Heart_Rate', 'Body_Temp'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.12801187)

In [6]:
df_train.groupby(['Duration', 'Heart_Rate'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.14161234)

In [7]:
df_train.groupby(['Duration', 'Body_Temp'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.1881298)

In [8]:
df_train.groupby(['Duration'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.17675276)

In [9]:
df_train.groupby(['Sex', 'Duration', 'Heart_Rate', 'Body_Temp'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.10964416)

In [10]:
df_train.groupby(['Height', 'Weight'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.9575898)

In [11]:
df_train.groupby(['Height'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.9839047)

In [12]:
df_train.groupby(['Weight'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(1.0028571)

In [13]:
df_train.groupby(['Sex', 'Height', 'Weight'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.9559728)

In [14]:
cc = sgpp.CatCombiner2(
    [['Sex', 'Duration', 'Heart_Rate', 'Body_Temp'], ['Sex', 'Height', 'Weight']]
)
df_train = dproc.join_and_assign(
    df_train, cc.fit_transform(df_train)
)

- Age를 제외하고 log 속성을 추가하면 성능 향상이 있습니다.

In [16]:
sgml.cv(df_train, kf, {
    'X_std': ['Duration_log', 'Heart_Rate_sqrt_d', 'Height_log', 'Weight_log', 'Heart_Rate_log', 'Body_Temp_log'] + X_num, 
    'X_tgt': df_train.columns[-2:].tolist(), 'tgt': {'smooth': 0.1, 'random_state': 123}}, config, lr_adapter)

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

{'valid_scores': [np.float64(0.09129340635479342),
  np.float64(0.09109195748206424),
  np.float64(0.09051232885890394),
  np.float64(0.09165227257363966)],
 'valid_prd': id
 0         4.976757
 1         3.613450
 2         3.350551
 3         4.915635
 4         5.010252
             ...   
 749995    5.359852
 749996    4.598993
 749997    5.459991
 749998    4.642135
 749999    4.609905
 Length: 750000, dtype: float64,
 'model_result': [],
 'train_scores': [np.float64(0.08861555140140004),
  np.float64(0.08846257710299021),
  np.float64(0.08872304101917802),
  np.float64(0.08852086168446736)],
 'hparams': {'X_std': ['Duration_log',
   'Heart_Rate_sqrt_d',
   'Height_log',
   'Weight_log',
   'Heart_Rate_log',
   'Body_Temp_log',
   'Age',
   'Height',
   'Weight',
   'Duration',
   'Heart_Rate',
   'Body_Temp'],
  'X_tgt': ['Sex__Duration__Heart_Rate__Body_Temp', 'Sex__Height__Weight'],
  'tgt': {'smooth': 0.1, 'random_state': 125}}}

In [4]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(5, random_state = 123, shuffle = True)

In [15]:
df_train

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,const,duration_bin,...,Heart_Rate_div_Body_Temp_sqrt,Body_Temp_div_Age_sqrt,Body_Temp_div_Height_sqrt,Body_Temp_div_Weight_sqrt,Body_Temp_div_Duration_sqrt,Body_Temp_div_Heart_Rate_sqrt,Body_Temp_sqrt,Calories_Log,Sex__Duration__Heart_Rate__Body_Temp,Sex__Height__Weight
id,,,,,,,,,,,,,,,,,,,,,
0,True,36,189.0,82.0,26.0,101.0,41.000000,150.0,1,8,...,15.773550,6.833333,2.982311,4.527692,8.040762,4.079653,6.403124,5.017280,True__26.0__101.0__41.0__,True__189.0__82.0__
1,False,64,163.0,60.0,8.0,85.0,39.700001,34.0,1,2,...,13.490364,4.962500,3.109544,5.125248,14.036070,4.306070,6.300794,3.555348,False__8.0__85.0__39.7__,False__163.0__60.0__
2,False,51,161.0,64.0,7.0,84.0,39.799999,29.0,1,1,...,13.314895,5.573115,3.136679,4.975000,15.042987,4.342535,6.308724,3.401197,False__7.0__84.0__39.8__,False__161.0__64.0__
3,True,20,192.0,90.0,25.0,105.0,40.700001,140.0,1,8,...,16.458569,9.100797,2.937269,4.290157,8.140000,3.971913,6.379655,4.948760,True__25.0__105.0__40.7__,True__192.0__90.0__
4,False,38,166.0,61.0,25.0,102.0,40.599998,146.0,1,8,...,16.008003,6.586189,3.151171,5.198297,8.120000,4.019999,6.371813,4.990433,False__25.0__102.0__40.6__,False__166.0__61.0__
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,True,28,193.0,97.0,30.0,114.0,40.900002,230.0,1,9,...,17.825560,7.729374,2.944047,4.152766,7.467284,3.830636,6.395311,5.442418,True__30.0__114.0__40.9__,True__193.0__97.0__
749996,False,64,165.0,63.0,18.0,92.0,40.500000,96.0,1,5,...,14.456405,5.062500,3.152921,5.102520,9.545942,4.222417,6.363961,4.574711,False__18.0__92.0__40.5__,False__165.0__63.0__
749997,True,60,162.0,67.0,29.0,113.0,40.900002,221.0,1,9,...,17.669195,5.280168,3.213408,4.996731,7.594940,3.847548,6.395311,5.402678,True__29.0__113.0__40.9__,True__162.0__67.0__
